# WGCNA
---

Standard WGCNA on a compact dataset...


## Install WGCNA
If you already have WGCNA installed on your system, then un-comment and execute the following lines:

In [ ]:
#source("http://bioconductor.org/biocLite.R") 
#biocLite(c("AnnotationDbi", "impute", "GO.db", "preprocessCore")) 
#install.packages('WGCNA', repos='http://cran.us.r-project.org')

## Load dependencies

In [ ]:
require(WGCNA)
require(dynamicTreeCut)

## Load expression data
This example uses synthetic expression data produced by the DREAM5 network inference effort (http://www.nature.com/nmeth/journal/v9/n8/full/nmeth.2016.html). The dataset consists of 805 arrays and 1632 genes. Sample labels (s1 through s805) were manually added. Note that the data file must reside in the same folder as the Jupyter notebook.

In [ ]:
dataFile <- "DREAM5_GeneExpressionNetwork1SilicoData.tsv"
datExpr <- read.table(dataFile, sep="\t", row.names=1, header=T)

# Transpose data frame
t_datExpr <- t(datExpr)

# Preview expression data frame - check that genes are in columns and 
# samples in rows
head(t_datExpr)

## Set WGCNA parameters

In [ ]:
beta <- c(3, 4)
beta <- c(4)
networkType <- "signed"
tomType <- "signed"
mergeCutHeight <- 0.15

# housekeeping parameter for directory creation
DATA_FILE_PATH = './'

## Run WGCNA

In [ ]:
for (b in beta) {
    
    print("LOG: Starting WGCNA")

    power <- b
    dset <- paste("synthetic", b, sep="_p")
    
    # create output subdirectories
    subDir <- paste("output", dset, sep="_")
    ifelse(!dir.exists(file.path(DATA_FILE_PATH, subDir)), dir.create(file.path(DATA_FILE_PATH, subDir)), FALSE)
    dset <-paste(subDir, dset, sep="/")

    ## dissimilarity matrix
    simMatrix <- NULL

    simMatrix <- TOMsimilarityFromExpr(t_datExpr, networkType=networkType, power=power)

    collectGarbage() ## WGCNA garbage collection

    row.names(simMatrix) <- row.names(datExpr)
    colnames(simMatrix) <- row.names(datExpr)

    dissMatrix <- 1 - simMatrix

    ## cluster
    geneTree <- hclust(as.dist(dissMatrix))

    ## isolate modules by cutting the dendrogram

    modules <- cutreeDynamic(geneTree, distM=dissMatrix)

    collectGarbage() ## WGCNA garbage collection

    modules <- labels2colors(modules) ## assign colors to modules

    ## calculate eigengenes
    ME <- moduleEigengenes(t_datExpr, colors=modules)
    eigengenes <- ME$eigengenes

    ## merge close modules
    merge <- mergeCloseModules(
        t_datExpr, 
        modules, 
        cutHeight = mergeCutHeight, 
        verbose = 1
    )
    modules.merged <- merge$colors
    eigengenes.merged <- merge$newMEs

    collectGarbage() ## WGCNA garbage collection

    ## Plot merged eigengene network
    print("LOG: Saving merged eigengene network...")
    png(paste(dset, "standard-eigengene-network.png", sep="-"))
    plotEigengeneNetworks(
        eigengenes.merged, 
        paste("Eigengene Network:",
        dset)
    )
    dev.off()

    ## Plot dendrogram with colors
    print("LOG: Saving dendogram...")
    png(paste(dset, "dendrogram.png", sep="-"))
    plotDendroAndColors(
        geneTree, 
        cbind(modules, modules.merged), 
        c("Modules", "After Merge"), 
        dendroLabels = FALSE, 
        hang = 0.03, 
        addGuide = TRUE, 
        guideHang = 0.05, 
        main=paste(dset, "module assignments")
    )
    dev.off()

    collectGarbage()

    ## Write module membership
    print("LOG: Saving module membership...")
    membership <- data.frame(modules=modules, merged_modules=modules.merged, row.names = row.names(datExpr))
    write.table(membership, paste(dset, "membership.txt", sep="-"), sep="\t", quote=F)

    ## save image
    print("LOG: Saving workspace...")
    save.image(paste(dset, ".RData.gz", sep=""), compress="gzip")
    
    print("LOG: Finished")
    print("------------------------------------------------------")

}


## Results

See the output directory for all output. Plots are also shown below:

In [ ]:
library("IRdisplay")

dendogram_png <- paste(dset, "dendrogram.png", sep="-")
display_png(file=dendogram_png)

dendogram_png <- paste(dset, "standard-eigengene-network.png", sep="-")
display_png(file=dendogram_png)  